## *Download data of food-*[101]

In [11]:
import tensorflow_datasets as tfds

# Automatically handles downloading and preprocessing
builder = tfds.builder('food101')
builder.download_and_prepare()  # Downloads in manageable chunks

# Access data
train_data = builder.as_dataset(split='train')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.LEJ9PV_2.0.0/food101-train.tfrecord*...:   0%|         …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.LEJ9PV_2.0.0/food101-validation.tfrecord*...:   0%|    …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [14]:
from google.colab import drive
drive.mount('/content/drive')

# Copy dataset to Drive (one-time)
!cp -r /root/tensorflow_datasets/food101 /content/drive/MyDrive/datasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!cp -r /content/drive/MyDrive/datasets/food-101/ /root/tensorflow_datasets/


Verify the Dataset is Properly Cached

In [42]:
import tensorflow_datasets as tfds
import tensorflow as tf

# List available datasets (should include 'food101')
print(tfds.list_builders())

# Check if food101 exists in cache
builder = tfds.builder('food101')
print(builder.info)  # Will show dataset details if loaded correctly

['abstract_reasoning', 'accentdb', 'aeslc', 'aflw2k3d', 'ag_news_subset', 'ai2_arc', 'ai2_arc_with_ir', 'ai2dcaption', 'aloha_mobile', 'amazon_us_reviews', 'anli', 'answer_equivalence', 'arc', 'asimov_dilemmas_auto_val', 'asimov_dilemmas_scifi_train', 'asimov_dilemmas_scifi_val', 'asimov_injury_val', 'asimov_multimodal_auto_val', 'asimov_multimodal_manual_val', 'asqa', 'asset', 'assin2', 'asu_table_top_converted_externally_to_rlds', 'austin_buds_dataset_converted_externally_to_rlds', 'austin_sailor_dataset_converted_externally_to_rlds', 'austin_sirius_dataset_converted_externally_to_rlds', 'bair_robot_pushing_small', 'bc_z', 'bccd', 'beans', 'bee_dataset', 'beir', 'berkeley_autolab_ur5', 'berkeley_cable_routing', 'berkeley_fanuc_manipulation', 'berkeley_gnm_cory_hall', 'berkeley_gnm_recon', 'berkeley_gnm_sac_son', 'berkeley_mvp_converted_externally_to_rlds', 'berkeley_rpt_converted_externally_to_rlds', 'big_patent', 'bigearthnet', 'billsum', 'binarized_mnist', 'binary_alpha_digits', 'b

Load the Dataset

In [40]:
(train_data, test_data), info = tfds.load(
    'food101',
    split=['train', 'validation'],
    as_supervised=True,
    with_info=True,
)
# Get class names (101 food categories)
class_names = info.features['label'].names
print(f"Class names: {class_names[:5]}")  # e.g., ['apple_pie', 'baby_back_ribs', ...]

Class names: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare']


# Inspect a few samples


In [29]:
for image, label in train_data.take(3):  # Unpack directly as (image, label)
    print(f"Image shape: {image.shape}, Label: {label}")

Image shape: (512, 512, 3), Label: 90
Image shape: (512, 512, 3), Label: 3
Image shape: (512, 512, 3), Label: 93


In [32]:
class_names = info.features['label'].names
print(f"Number of classes: {len(class_names)}")
print(f"First 5 classes: {class_names[:101]}")

Number of classes: 101
First 5 classes: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'mac

In [33]:
print(f"Training samples: {info.splits['train'].num_examples}")
print(f"Validation samples: {info.splits['validation'].num_examples}")

Training samples: 75750
Validation samples: 25250


In [36]:
# Check shapes and labels (no plots)
for image, label in train_data.take(3):
    print(f"Image shape: {image.shape}, Label ID: {label}, Class name: {class_names[label]}")

Image shape: (), Label ID: 56, Class name: huevos_rancheros
Image shape: (), Label ID: 76, Class name: pizza
Image shape: (), Label ID: 21, Class name: chocolate_cake


In [37]:
print(f"Image dtype: {image.dtype}, Label dtype: {label.dtype}")

Image dtype: <dtype: 'string'>, Label dtype: <dtype: 'int64'>


Resize Image

In [43]:
# 3. Define preprocessing function
def preprocess(image, label):
    # Convert bytes to tensor if needed (handled automatically by tfds)
    image = tf.image.resize(image, [224, 224])  # Resize
    image = tf.cast(image, tf.float32) / 255.0  # Normalize [0,1]
    return image, label

# 4. Apply preprocessing
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

# 5. Batch and optimize pipeline
BATCH_SIZE = 32
train_data = train_data.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# 6. Verify preprocessing
for images, labels in train_data.take(1):
    print(f"Batch image shape: {images.shape}")  # Should be (32, 224, 224, 3)
    print(f"Batch labels shape: {labels.shape}") # Should be (32,)
    print(f"Sample label: {labels[0].numpy()} - {class_names[labels[0].numpy()]}")

Batch image shape: (32, 224, 224, 3)
Batch labels shape: (32,)
Sample label: 57 - hummus


# Building and Training Models on Food-101

In [ ]:
from tensorflow.keras import layers, models

def build_cnn():
    model = models.Sequential([
        # Conv Block 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Conv Block 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Conv Block 3
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),

        # Classifier
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(101, activation='softmax')
    ])

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    return model

cnn_model = build_cnn()
cnn_model.summary()